In [ ]:
import numpy
from math import pi
from matplotlib import pyplot, mlab
%matplotlib inline
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

def Joukowsky(etha, a):
    z = a/2. * (etha + 1/etha)

    return z

def circulo(centro, R, N):
    alpha = numpy.linspace(0,2*pi,N)

    x_prime = R*numpy.cos(alpha)
    y_prime = R*numpy.sin(alpha)
    
    etha = numpy.array(x_prime+1j*y_prime)
    
    etha += centro
    
    return etha
    

N = 101
centro_x = -0.1
centro_y = 0.    

# Radio para que circulo intersecte punto (1,0)
# Ojo: el punto (-1,0) debiese queda DENTRO del circulo (no lo checkeo aca)
R = numpy.sqrt((1-centro_x)**2+centro_y**2)

# Tamaño del perfil
a = 1

centro = centro_x+centro_y*1j

# Generar cilindro
etha = circulo(centro, R, N)

x_cil = numpy.real(etha)
y_cil = numpy.imag(etha)

pyplot.figure()

fig=pyplot.plot(x_cil,y_cil,c='k')
fig2=pyplot.plot(0, 0, marker='+', markersize=10, c='k')
fig3=pyplot.plot(centro_x,centro_y,marker='o', markersize=5, c='k')
pyplot.axis('equal')
pyplot.xlim([-(R+numpy.abs(centro))-0.1,(R+numpy.abs(centro))+0.1])
pyplot.ylim([-(R+numpy.abs(centro))-0.1,(R+numpy.abs(centro))+0.1])

In [ ]:
# Generar perfil de Joukouski
z = Joukowsky(etha, a)

x = numpy.real(z)
y = numpy.imag(z)

pyplot.clf()
pyplot.figure()
fig=pyplot.plot(x,y,c='k')
pyplot.axis('equal')
pyplot.xlim([-a-0.1,a+0.1])
pyplot.ylim([-a-0.1,a+0.1])

In [ ]:
nx = 100
L = 6.0
xp = numpy.linspace(-L/2,L/2,nx)
Xp,Yp = numpy.meshgrid(xp,xp)

Zp = Xp + Yp*1j

alpha_at = 10*pi/180.

# Flujo uniforme
U_inf = 1.
PHI_U = U_inf*Zp*numpy.exp(-1j*alpha_at)

# Doblete (corrido a 'centro')
K= R**2*U_inf
PHI_d = K/(Zp-centro)*numpy.exp(-1j*alpha_at)

# Vortice (corrido a 'centro')
Gamma = 2.
PHI_v = 1j*Gamma/(2*pi)*numpy.log(Zp-centro)

# Constante extra para que psi=0 sea siempre el cilindro
PHI_c = PHI_U + PHI_d + PHI_v #- 1j*Gamma/(2*pi)*numpy.log(R-numpy.abs(centro))

psi_c = numpy.imag(PHI_c)
phi_c = numpy.real(PHI_c)

contours = numpy.arange(-4,4,0.5)

pyplot.figure()
C = pyplot.contour(Xp, Yp, psi_c, contours, colors='k')
C2 = pyplot.plot(x_cil,y_cil,c='r')
pyplot.clabel(C, inline=1, fontsize=10)

pyplot.axis('equal')
pyplot.xlim([-L/2,L/2])
pyplot.ylim([-L/2,L/2])


In [ ]:
from scipy import interpolate

# Conviene pasar Zp y psi a 1D (por numpy.where y mlab.griddata)
Zp_1D = numpy.ravel(Zp)
psi_c_1D = numpy.ravel(psi_c)

# Los puntos dentro del cilindro no tienen significado fisico y los sacamos
fuera_cil = numpy.where(numpy.abs(Zp_1D-centro)>R)[0]

psi_perfil = psi_c_1D[fuera_cil]

Z = Joukowsky(Zp_1D[fuera_cil], a)
X = numpy.ravel(numpy.real(Z))
Y = numpy.ravel(numpy.imag(Z))


# pyplot.countour funciona solamente con mallas regulares
# Interpolar X,Y a malla regular
xi= numpy.linspace(-L/2,L/2,200)
Xi,Yi = numpy.meshgrid(xi,xi)

psi_i = mlab.griddata(X, Y, psi_perfil, Xi, Yi, interp='linear')


pyplot.clf()
pyplot.figure()
contours = numpy.arange(-4,4,0.3)

C=pyplot.contour(Xi, Yi, psi_i, contours, colors='k')
C2=pyplot.plot(x,y,c='r')
#pyplot.clabel(C, inline=1, fontsize=10)

pyplot.axis('equal')
pyplot.xlim([-a-0.1,a+0.1])
pyplot.ylim([-a-0.1,a+0.1])